### <$Q_\pi$를 추정하기 위한  비활성 정책 n 단계 살사> 

입력 : 모든 $s \in S, a \in A$에 대해 $b(a|s)$을 만족하는 행동 정책 b를 임의의 값으로 설정  
모든 $s \in S, a \in Q$ 에 대해 Q(s,a)를 임의의 값으로 초기화 
정책 $\pi$를 Q에 대해 탐욕적인 정책이 되도록 초기화, 또는 주어진 고정된 정책으로 초기화 
알고리즘 파라미터 : 시간 간격 $\alpha \in (0,1] $, 양의 정수 $\alpha$  
($S_t$와 $A_t$, $R_t$에 대한) 모든 저장과 접근은 n+1로 인텍스를 설정하여 행해질 수 있음. 

각 에피소드에 대한 루프 : 
- $S_0$를 초기화하고 저장(단, $S_0 \neq$ 종단) 
- 행동 $A_0 ~ b(.|S_a)$를 선택하고 저장  
- $ T <- $ 무한대
- t= 0,1,2, ... 에 대한 루프 : 

> t < T 이면 : 
 > -  행동 $A_t$를 취함
 > - 다음 보상을 $R_{t+1}$, 다음 상태를 $S_{t+1}$로 측정하고 저장함 
 > - $S_{t+1}$ 이 종단이면, T <- t+1 
 > - 종단이 아니면, 행동 $A_{t+1} ~ $\b(.|S_{t+1})을 선택하고 저장 
 
> r <- t-n+1(r은 상태의 추정값이 갱신되는 시각) 

> r>=0 이면 : 
> - $\rho$ <- $\prod^{min(r+n, T-1)}_{i=r+1} \frac{\pi(A_i|S_i)}{b(A_i|S_i)}$
> - G <- $\sum^{min(r+n,T)}_{i = r+1} \gamma^{i-r-1} R_i$ 
> - r+n < T 이면, G <- $G+ \gamma^n Q(S_{r+n}, A_{t+n})$
> - $Q(S_r, A_r)$ <- $V(S_r, A_r) + \alpha [G-V(S_r, A_r)]$
> - $\pi$를 학습한다면 $\pi(.|S_r)$ 가 Q에 대해 입실론 탐욕적이 되도록 해야 함. 

- r = T-1 이면 종료


**<구현해야 하는 것>**
- b(s,a) : 행동 정책 

**<필요한 것>** 
- $\alpha$ : class 제작시 입력 값으로 부여 
- n : 몇단계 TD를 할 것인가? 

**<함수 / 데이터 형식>** 
- class evaluate_Q_n_TD : 
> def __init__(self, S, A, n, b, alpha, reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 

**<외부함수>** 
- R(s',a,s) : 보상함수
- b(s) : 상태 s에서 선택할 행동들의 확률 값 

**<고민점>** 


In [1]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [2]:
from collections import defaultdict
import random
import numpy as np

In [3]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 

def choose_random_max(lst) : #최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a의 인덱스를 추출 
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 


def b(s) : # 모든 행동을 공평하게 뽑는 정책 
    return [1/10]*10 #


In [34]:
# n 단계 Q값 추정 코드 참고 

class off_policy_evaluate_Q_n_TD :  
    def __init__(self, S, A, n, b, reward_func, alpha=0.1,  epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.n = n
        self.b = b
        self.reward_func = reward_func
        self.alpha = alpha
        self.epsilon = epsilon 
        self.gamma = gamma 
        self.num_episode = num_episode
        self.T = len_episode

        self.Q = self.initiate_Q() 
#        self.V = self.initiate_V() 
    
    def initiate_Q(self) : # Q(s,a) 값을 초기화 
        Q_dict = defaultdict(float)
        for s in self.S : 
            for a in self.A : 
                Q_dict[(s,a)] = 0         
        
        return Q_dict

    def pi(self, s) : #Q 값을 기반으로 하여 "탐욕적" 선택하도록 확률 값을 반환하기.  
        lst = [] 
        for a in self.A : lst.append(self.Q[(s,a)])
        max_a = choose_random_max(lst)
        prob_s = [0]*len(self.A) 
        prob_s[max_a] = 1 
        return prob_s
    
    def choice_action(self, s, policy) : #수정 - pi 함수를 재정의한 것과 맞춰 수정 
        a= random.choices(self.A, weights = policy(s))
        a = a[0]
        return a
    
    def next_s(self, s,a) : # 상태 s에서 a 행동을 했을 때 다음 상태 s'. 정책, S,A 에 따라 달라짐. 
        return min(max(s+a, 0), max(self.S)) 


    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []} 
        episode["R"].append(0) # R_0 값 부여
        episode["S"].append(start_s) #S_0 값 부여.
        for _ in range(T) : 
            episode["S"].append(s)
            a = self.choice_action(s, self.b) 
            next_s = self.next_s(s, a)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # 
        
        for _ in range(self.num_episode) : 
            start_s = random.choice(self.S) 
            T = self.T # 어차피 T는 self.T로 갱신되기 때문에 앞에서 설정함. 
            episode = self.make_episode(start_s, self.T) 
            S,R,A = episode['S'],episode['R'], episode['A'] 
            
            for t in range(self.T + self.n +1) : # t는 아무리 커도 self.T + n 보다 커질 수 없기 때문 
                # make_episode 에서 이미 a,r를 계산해 두었기 때문에 t<T 조건문 수도코드 구현 생략. 
                
                # r 또한 현재 t와 인덱스로 사용되는 n이 중복되기 때문에, t값이 T 값일 때 순환문을 종료하는 것으로 
                
                r = t - self.n +1 
                if r >= 0 : 
                    rho = 1 
                    rho_lst = [self.pi(S[i])[self.A.index(A[i])] / self.b(S[i])[self.A.index(A[i])] for i in range(r+1, min(r+self.n, self.T))]
                    for p in rho_lst : rho *= p
                    G = sum([self.gamma **(i-r-1) * R[i] for i in range(r+1, min(r+self.n, self.T)+1)])
                    if r+self.n < self.T : G = G + self.gamma**self.n * self.Q[(S[r+self.n], A[r+self.n])]
                    self.Q[S[r], A[r]] = self.Q[S[r], A[r]] + self.alpha * rho* (G - self.Q[S[r], A[r]])
                if r == self.T -1 : break                  
                    
        

In [35]:
test = off_policy_evaluate_Q_n_TD(S,A, 10, b, reward_func)
test.update_returns() 
#print(test.Q)

for _ in range(1000) : test.update_returns()  
print(test.Q)

# 확실히 비활성 정책으로 하는게 여러가지 경우를 포함하면서 학습을 잘하는 듯 

defaultdict(<class 'float'>, {(0, -5): 0.205, (0, -4): -0.493, (0, -3): -0.47616333333333327, (0, -2): -0.17195000000000002, (0, -1): -0.983340163, (0, 0): 0.0, (0, 1): -0.6355000000000001, (0, 2): 0.5343368382029251, (0, 3): -0.19211856509283332, (0, 4): 0.3662261467, (1, -5): 0.12804999999999997, (1, -4): 0.11855, (1, -3): 0.15213754230716658, (1, -2): -0.32566077995000003, (1, -1): -0.09829588006989551, (1, 0): 0.0, (1, 1): -0.913906558, (1, 2): 0.67433922005, (1, 3): -0.24860447238903333, (1, 4): 0.13042577500000002, (2, -5): 0.010426373449278586, (2, -4): -0.76354, (2, -3): -0.003285409976366671, (2, -2): -0.4624763411757502, (2, -1): 0.5370109738797499, (2, 0): 0.2657205, (2, 1): -0.6355000000000001, (2, 2): 1.1561, (2, 3): 0.5666666666666667, (2, 4): 0.6550000000000001, (3, -5): -5.1041300000000005, (3, -4): -0.5200000000000007, (3, -3): 1.1433333333333335, (3, -2): -1.0560999999999998, (3, -1): 1.7290000000000003, (3, 0): 0.0, (3, 1): -0.73, (3, 2): 0.614383962274805, (3, 3): 0